<a href="https://colab.research.google.com/github/ish66726-a11y/colab-notebooks/blob/main/notebooks/02%E5%82%B5%E5%88%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#2 債券

##2-1 債券の基礎

### 債券とは
- 投資家が企業や政府にお金を貸すことで成立する「負債投資」。
- 発行体は一定期間資金を借り、利子を支払う。

---

### 債券の種類

#### 1. ゼロクーポン債（Zero Coupon Bond）
- 元本のみ返済、利子なし。
- 価格（現在価値）:
  $$P = \frac{X}{(1+r)^N}$$
  - P：債券価格（現在価値）
  - X：額面（満期時に受け取る元本）
  - r：市場金利（1期あたりの割引率）
  - N：満期までの期数（年1回複利なら年数）
- 例：額面 1000円、満期 2年、市場金利 4%
  → 価格  ≒ 924円

#### 2. クーポン債（Coupon Bond）
- 毎年クーポン（利子）を支払い、満期時に元本も返済。
- 価格（現在価値）:
  $$P = \sum_{t=1}^N \frac{C}{(1+r)^t} + \frac{X}{(1+r)^N}$$
  または  
  $$P = \frac{C}{r}\left(1 - \frac{1}{(1+r)^N}\right) + \frac{X}{(1+r)^N}$$
  - P：債券価格（現在価値）
  - C：各期に支払われるクーポン額  
    $(C = \text{額面} \times \text{クーポン率})$
  - X：額面（満期時に返済される元本）
  - r：市場金利（1期あたりの割引率）
  - N：満期までの期数
  - t：クーポン支払時点を表すインデックス
- 例：額面 1000円、クーポン率 10%、満期 3年、市場金利4%
  → 価格 ≒ 1167円

---

👉 債券価格は「将来キャッシュフローの現在価値の合計」で決まる。  
👉 計算時は **市場金利** を使う点が重要。

---

### Python実装例
（下のコードセル参照）

In [ ]:
#ゼロクーポン債を表すクラス
class ZeroCouponBond:
    def __init__(self, principal, maturity, interest_rate):
        self.principal = principal                # 額面（満期に返ってくる元本）
        self.maturity = maturity                  # 満期年数
        self.interest_rate = interest_rate / 100  # クーポン利率（% → 小数へ変換）

#現在価格PVを計算するメソッド
    def present_value(self, X, n):
        return X / ((1 + self.interest_rate) ** n)

#ゼロクーポン債の価格を計算するメソッド
    def calculate_price(self):
        return self.present_value(self.principal, self.maturity)

#実行部分
if __name__ == "__main__":
    bond = ZeroCouponBond(1000, 2, 4)  # 元本1000, 満期2年, 金利4% の債券を作成
    price = bond.calculate_price()
    print(f"債券価格 = {price:.2f}円")

債券価格 = 924.56ドル


In [ ]:
#クーポン債を表すクラス
class CouponBond:
    def __init__(self, principal, rate, maturity, interest_rate):
        self.principal = principal              # 額面（満期に返ってくる元本）
        self.rate = rate / 100                  # クーポン利率（% → 小数へ変換）
        self.maturity = maturity                # 満期年数
        self.interest_rate = interest_rate / 100  # 市場金利（割引率）

    # 将来受け取る金額 X を n年後の現在価値に割り引くメソッド
    def present_value(self, X, n):
        return X / ((1 + self.interest_rate) ** n)

    # クーポン債の価格を計算するメソッド
    def calculate_price(self):
        price = 0                                # 債券価格を入れる変数を初期化
        coupon = self.principal * self.rate      # 毎年支払われるクーポン（利息）

        # 1年目から満期まで、毎年のクーポンの現在価値を足していく
        for t in range(1, self.maturity + 1):
            price += self.present_value(coupon, t)
        price += self.present_value(self.principal, self.maturity) # 最後に満期時の元本（principal）の現在価値を足す
        return price


# 実行部分
if __name__ == "__main__":
  bond = CouponBond(1000, 10, 3, 4)# 元本1000、クーポン10%、満期3年、市場金利4% の債券を作成
  price = bond.calculate_price()
  print(f"債券価格 = {price:.2f}円")

債券価格 = 1166.51ドル


##2-2 債券の利回りと最終利回り（YTM）

### 利回り（Yield）
- 投資収益を示す指標。
- 計算式：
  $$\text{Yield} = \frac{\text{年間クーポン支払額}}{\text{債券価格}}$$
- 例：価格 1,167円、年間クーポン 100円 → Yield ≒ 8.6%

---

### 最終利回り（Yield to Maturity, YTM）
- 債券を満期まで保有した場合の「実質利回り」。
- 内部収益率(IRR) に相当。
### 最終利回り（Yield to Maturity, YTM）

**定義（離散時間・年1回複利）**

$$
P = \sum_{t=1}^{N} \frac{C}{(1+y)^t} + \frac{X}{(1+y)^N}
$$

- $P$：債券価格（現在の市場価格）
- $C$：各期に支払われるクーポン額  
  （$C = \text{額面} \times \text{クーポン率}$）
- $X$：額面（満期時に返済される元本）
- $y$：最終利回り（YTM、1期あたりの利回り）
- $N$：満期までの期数（年1回払いなら年数）
- $t$：クーポン支払時点を表すインデックス

---

**定義（連続時間モデル・連続複利）**

$$
P
= \sum_{i=1}^{n} C_i \, e^{-y (t_i - t)}
+ X \, e^{-y (T - t)}
$$
- $P$：債券価格（評価時点 $t$ における現在価値）
- $C_i$：時点 $t_i$ に支払われるクーポン額  
  （一定クーポンなら $C_i = C$）
- $X$：満期時点 $T$ に支払われる元本（額面）
- $y$：連続複利ベースの最終利回り（YTM）
- $t$：評価時点（現在）
- $t_i$：第 $i$ 回目のクーポン支払時点
- $T$：満期時点
- $n$：クーポン支払回数


- YTM は「債券価格と将来キャッシュフローの現在価値を一致させる割引率」であり、一般に解析的に解けないため、数値計算（ニュートン法など）で求められる。


---

### イールドカーブ（Yield Curve）
- 縦軸：利回り、横軸：満期（年数）。
- 通常は右上がりの形状をとる。
- 理由：
  - 将来の短期金利上昇に対する市場の期待
  - 長期債に固有の期間プレミアム（価格変動・流動性リスク）
  - インフレ不確実性


---

👉 Yield = 単純利回り、YTM = 実際の内部収益率。  
👉 イールドカーブは「期間が長いほど利回りが高い」傾向を示す。


##2-3 市場金利と債券価格の関係
### 基本原則
- 債券価格と市場金利は **逆相関**。
  - 金利 ↑ → 債券価格 ↓
  - 金利 ↓ → 債券価格 ↑

---

#### ゼロクーポン債
- 価格：
  $$P = \frac{X}{(1+r)^N}$$
#### クーポン債
- 価格：
  $$P = \sum_{t=1}^N \frac{C}{(1+r)^t} + \frac{X}{(1+r)^N}$$
- 両方とも分母に金利$r$があるため、金利上昇 → 債券価格下落。

---

### 金利リスク（Interest Rate Risk）
- 金利が上昇すると、割引率が高くなり債券価格は下落する。
- 金利が低下すると、割引率が低くなり債券価格は上昇する。
- これは、債券価格が将来キャッシュフローの現在価値で決まるためである。


---

👉 債券価格は「市場金利と逆相関」する。  
👉 これが金利リスクの背景である。


##2-4 マコーレー・デュレーションとは

- 債券の **キャッシュフロー（クーポン＋元本）** によって、投資額が回収されるまでの平均的な時間。
- 「お金がいつ戻ってくる債券か」を **時間** で表した指標。
- 債券価格の **金利感応度（利回り変化への敏感さ）** を測るために使われる。

---

## 数学的定義

債券価格 $P$ を満期利回り $y$ の関数とするとき、  
マコーレー・デュレーション $D$ は次で定義される：

$$
D = -\frac{1}{P}\frac{dP}{dy}
$$

- $P$：債券価格  
- $y$：満期利回り（YTM）

👉 「金利が少し変わったとき、債券価格がどれだけ変わるか」を時間に変換したもの

---

## ゼロクーポン債のデュレーション

- ゼロクーポン債は途中のキャッシュフローが存在しない。
- 満期時に元本のみを一括で受け取る。

### 結論
- マコーレー・デュレーション ＝ 満期までの時間

**理由：**
- キャッシュフローが「満期の1点」にしか存在しないため  
  → 平均回収時間も満期そのものになる。

---

## クーポン債のデュレーション

- クーポン債では、毎期クーポン＋満期時の元本返済が存在。
- 各キャッシュフローの「現在価値」を重みとして、回収時間の平均を取る。

### 直感的な理解
- 早く返ってくるお金ほど、デュレーションを短くする
- 満期が長いほど、デュレーションは長くなる

---

## デュレーションと金利リスク

- 債券価格と市場金利は **逆の関係**。
- デュレーションが長いほど、金利変動に対する価格変動が大きい。

| デュレーション | 金利変動への感度 |
|---------------|----------------|
| 短い | 小さい |
| 長い | 大きい |

---

## 投資判断との関係

- 金利が **下がる** と予想される場合  
  → **デュレーションの長い債券** が有利
- 金利が **上がる / 横ばい** と予想される場合  
  → **デュレーションの短い債券** が有利

👉 マコーレー・デュレーションは 金利リスクを管理し、適切な債券を選ぶための基本指標

---

## まとめ

- マコーレー・デュレーションは「債券価格がキャッシュフローで回収されるまでの平均時間」
- ゼロクーポン債： デュレーション＝満期
- クーポン債：  クーポンと元本を考慮した加重平均時間
- 金利変動リスクの評価に不可欠


##2-5 債券のリスク

### 1. 金利リスク（Interest Rate Risk）
- 債券価格と市場金利は逆相関。
- 金利↑ → 債券価格↓
- 満期まで保有すれば価格変動による損益は確定せず、契約通りのキャッシュフローを受け取れるが、途中売却時に損失リスク。

---

### 2. デフォルトリスク（Default Risk）
- 発行体（企業・政府）が利息や元本を支払えないリスク。
- 例：クーポン（年間10000円）が支払われない可能性。

---

### 3. インフレリスク（Inflation Risk）
- インフレにより将来キャッシュフローの購買力が低下するリスク。
- 名目額を受け取っても実質価値は目減りする。

---

👉 債券投資では「金利リスク・デフォルトリスク・インフレリスク」の3つが特に重要。


##2-6 株式と債券の違い

| 項目 | 株式（Stocks） | 債券（Bonds） |
|---|---|---|
| 性質 | 企業の**所有権** | 政府・企業への**貸付（負債証券）** |
| 投資家の権利 | 議決権あり | 議決権なし |
| 収益の形 | 配当（支払われることがある） | 利息のみ（配当なし） |
| 元本返済 | なし | 満期時に元本返済 |
| リスク水準 | 高い | 低い |
| 価格特性 | 企業業績に依存 | 市場金利と逆相関 |
| 一般的評価 | ハイリスク・ハイリターン | ローリスク・安定収益 |
